<a href="https://colab.research.google.com/github/VondracekS/ExplainabilityExchange/blob/master/ExplainerDashboardDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explainer Dashboard Demo - penguin dataset

In [1]:
pip install explainerdashboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.9/286.9 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import plotly.express as px
from explainerdashboard import ExplainerDashboard, ClassifierExplainer
from seaborn import load_dataset

In [12]:
from sklearn.model_selection import train_test_split
data_cls = load_dataset("penguins")
data_reg = load_dataset("mpg")

for element in (data_cls, data_reg):
  print(element.name, element.shape, "\n", element.columns, "\n\n")

AttributeError: ignored

In [18]:
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(data_reg.drop('mpg', axis=1), data_reg[['mpg']])
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(data_cls.drop('species', axis=1), data_cls[['species']])

In [ ]:
# data_train.dtypes

species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
dtype: object

In [ ]:
# fig = px.scatter_matrix(data_train, color='species', template='ggplot2')
# fig.update_layout(autosize=False, width=1500, height=1000, title = "Penguins Scatter Matrix plot")

In [ ]:
# data_train.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [13]:
# data_train.dtypes

NameError: ignored

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

def create_pipeline(data):
  cat_features = [col for col in data.columns if data[col].dtype == 'object']
  num_features = [col for col in data.columns if data[col].dtype != 'object']

  numeric_transformer = Pipeline(
      steps=[('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
  ])
  categorical_transformer = Pipeline(
      steps=[
          ('imputer', SimpleImputer(strategy='constant')),
          ('encoder', OrdinalEncoder())]
  )
  preprocessor = ColumnTransformer(
      transformers=[
          ('numeric', numeric_transformer, num_features),
          ('categorical', categorical_transformer, cat_features)
      ]
  )

  pipeline = Pipeline(
      steps=[
          ('preprocessor', preprocessor),
          ('scaler', StandardScaler()), 
          ('encoder', OrdinalEncoder()),
          ('imputer', SimpleImputer(strategy="median"))
      ]
  )

  return pd.DataFrame(pipeline.fit_transform(data), 
                      columns=pipeline.get_feature_names_out())

  # train_transformed = pd.DataFrame(pipeline.fit_transform(data_train), columns=pipeline.get_feature_names_out())
  # test_transformed = pd.DataFrame(pipeline.fit_transform(data_test), columns=pipeline.get_feature_names_out())


In [20]:
create_pipeline(y_reg_test)

,numeric__mpg
0,54.0
1,28.0
2,26.0
3,28.0
4,12.0
...,...
95,13.0
96,37.0
97,35.0
98,28.0


In [ ]:
(X_train, X_test, y_train, y_test) = (train_transformed.drop('categorical__species', axis=1), 
                                      test_transformed.drop('categorical__species', axis=1),
                                      train_transformed['categorical__species'],
                                      test_transformed['categorical__species'])

In [22]:
(X_train, X_test, y_train, y_test) = (
    create_pipeline(X_reg_train), create_pipeline(X_reg_test), 
    create_pipeline(y_reg_train), create_pipeline(y_reg_test)
)

In [23]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
fit = model.fit(X_train, y_train)
preds = fit.predict(X_test)

In [26]:
from explainerdashboard import RegressionExplainer, ExplainerDashboard
explainer = RegressionExplainer(model, X_test, y_test)

Generating self.shap_explainer = shap.LinearExplainer(modelX)...


In [28]:
ExplainerDashboard(explainer, mode='external', index_dropdown=False).run()

Building ExplainerDashboard..
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://172.28.0.12:8050
You can terminate the dashboard with ExplainerDashboard.terminate(8050)
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

In [29]:
!pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
ansi2html==1.8.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
astropy==5.2.1
astunparse==1.6.3
attrs==22.2.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.6
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==39.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.33
dash==2.9.1
dash-auth==2.0.0
dash-bootstrap-components==1.4.1
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
dask==2022.12.1
dat